In [1]:
from glob import glob
import numpy as np
import pandas as pd
from scipy import stats

from analysis_scripts import second_harm_tools
from analysis_scripts.analyzs_many import get_human

/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
human_skeleton = glob("../data/human_skeleton/*")

In [3]:
human_skeleton

['../data/human_skeleton/Skeletonexpnt002Cell1',
 '../data/human_skeleton/Skeletonexpnt003Cell1',
 '../data/human_skeleton/Skeletonexpnt004Cell1',
 '../data/human_skeleton/Skeletonexpnt006Cell1',
 '../data/human_skeleton/Skeletonexpnt006Cell2Small',
 '../data/human_skeleton/Skeletonexpnt007Cell1_Small']

In [4]:
defaults =  {"t_range": slice(None, None)}
params = {"Skeletonexpnt006Cell2Small":{
             "t_range": slice(None,5500)},
          
        "Skeletonexpnt006Cell1":{
             "t_range": slice(None,8800)
            } 
         }


results = []
spectrums = []
for exp in human_skeleton:
    param = params.get(exp.split("/")[-1], defaults)
    
    if param.get("skip"):
        continue
    a = get_human(exp, rel_s=0.9, **param)
    if a.last_valid_index() < 0.1:
        continue
    res_human, spectrum = second_harm_tools.analyse_shape(a, NFFT=250, pca=False, smoothing=139)
    res_human.index.name = "time"
    res_human.reset_index(inplace=True)
    res_human["exp"] = exp
    results.append(res_human)
    spectrums.append(spectrum)

res_humans = pd.concat(results)


/Data/iff_th2/rode/sperm/experiments/sperm_compare_species/nb/analysis_scripts/second_harm_tools.py:159: RuntimeWarning: invalid value encountered in sqrt
  eig = np.sqrt(eig[::-1])
/Data/iff_th2/rode/sperm/experiments/sperm_compare_species/nb/analysis_scripts/second_harm_tools.py:161: RuntimeWarning: invalid value encountered in greater
  num_ev = np.cumsum(eig> thres).max()


2


/Data/iff_th2/rode/sperm/experiments/sperm_compare_species/nb/analysis_scripts/second_harm_tools.py:125: RuntimeWarning: divide by zero encountered in true_divide
  mode='full')/c0f,


2


KeyboardInterrupt: 

In [5]:
len(human_skeleton)

6

In [7]:
5./res_humans.dropna().omega.mean()*500

138.41946607595514

In [8]:
res_humans["phi_vel_normed_s_c0"] = res_humans.phi_vel_normed_s/(res_humans.C0**2)
res_humans["c2_plus_cm"] = (9/16/np.pi)*res_humans.C2_SIN + (np.pi**2-3)/4/np.pi**2*res_humans.kappa_mean_s

In [9]:
res = []
for spectrum in spectrums:
    res.append((spectrum[::500,:,0].flatten(), np.sqrt(spectrum[::500,:,1].flatten())))
xx = np.hstack(list(zip(*res))[0])
yy = np.hstack(list(zip(*res))[1])
    

In [10]:
power, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="mean")
power_std, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="std")

freq = (bin_edge[1:]+bin_edge[:-1])/2


/Local/rode/.conda/envs/py36/lib/python3.6/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


In [11]:
res_humans.to_hdf('../c2_kappa_mean_rotation.h5', 'humans')
np.savetxt("../human_spectrum.csv",  np.vstack([freq, power, power_std]).T)

/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['stuck', 'exp']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
